In [55]:
import pandas as pd
import numpy as np
import plotly.express as px
import locale
locale.setlocale( locale.LC_ALL, '')


'English_Canada.1252'

In [ ]:
def classify_frequency(days):
    if pd.isna(days):
        return 'One-time'
    elif days < 10:
        return 'Weekly'
    elif days < 18:
        return 'Bi-weekly'
    elif days < 36:
        return 'Monthly'
    else:
        return 'Occasional'
    



In [25]:
df = pd.read_csv('Sales_20250201_20250331.csv')
df.columns = df.columns.str.strip()
df.drop(columns=['Invoice Date', 'Subtotal', 'Patient', 'Location', 'Staff Member', 'Income Category', 'Payer', 'Details'], inplace=True)
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'], format='%Y-%m-%d')
print(df.shape)
df.head()

(112, 8)


,Purchase Date,Patient Guid,Item,Invoice #,Status,Total,Collected,Balance
0,2025-02-03,29276-67,Individual Counselling Session (Video),1457-P01,paid,157.5,157.5,0.0
1,2025-02-03,29276-79,Individual Counselling Session (Video),1458-P01,paid,157.5,157.5,0.0
2,2025-02-03,29276-90,Individual Counselling Session (Video),1459-P01,paid,157.5,157.5,0.0
3,2025-02-03,29276-40,Individual Counselling Session (Video),1460-P01,paid,157.5,157.5,0.0
4,2025-02-04,29276-64,Individual Counselling Session (Video),1461-P01,paid,168.0,168.0,0.0


In [3]:
monthly_details = df.groupby(df['Purchase Date'].dt.to_period('M')).agg({'Invoice #': 'count', 'Total': 'sum', 'Collected':'sum', 'Balance': 'sum'}).reset_index()
monthly_details['Total_avg'] = round(monthly_details['Total'] / monthly_details['Invoice #'], 2)
monthly_details['Collected_avg'] = round(monthly_details['Collected'] / monthly_details['Invoice #'], 2)
monthly_details['Balance_avg'] = round(monthly_details['Balance'] / monthly_details['Invoice #'], 2)
monthly_details.columns = ['Month', '# of Sessions', 'Total Charged', 'Total Collected', 'Toal Outstanding', 'Average Charged', 'Average Collected', 'Average Outstanding']
monthly_details

,Month,# of Sessions,Total Charged,Total Collected,Toal Outstanding,Average Charged,Average Collected,Average Outstanding
0,2025-02,67,9978.75,9752.75,226.0,148.94,145.56,3.37
1,2025-03,45,6885.00,6465.00,420.0,153.00,143.67,9.33


In [4]:
annual_details = df.groupby(df['Purchase Date'].dt.to_period('Y')).agg({'Invoice #': 'count', 'Total': 'sum', 'Collected':'sum', 'Balance': 'sum'}).reset_index()
annual_details['Total_avg'] = round(annual_details['Total'] / annual_details['Invoice #'], 2)
annual_details['Collected_avg'] = round(annual_details['Collected'] / annual_details['Invoice #'], 2)
annual_details['Balance_avg'] = round(annual_details['Balance'] / annual_details['Invoice #'], 2)
annual_details.columns = ['Year', '# of Sessions', 'Total Charged', 'Total Collected', 'Toal Outstanding', 'Average Charged', 'Average Collected', 'Average Outstanding']
annual_details

,Year,# of Sessions,Total Charged,Total Collected,Toal Outstanding,Average Charged,Average Collected,Average Outstanding
0,2025,112,16863.75,16217.75,646.0,150.57,144.8,5.77


In [33]:
client_details = df.groupby('Patient Guid', as_index=False).agg({'Invoice #': 'count', 'Total': 'sum', 'Collected':'sum', 'Balance': 'sum'}).reset_index(drop=True)
client_details.columns = ['Patient Guid', '# of Sessions', 'Total Charged', 'Total Collected', 'Total Outstanding']
client_avgs = df.groupby('Patient Guid', as_index=False).agg({'Total': 'mean', 'Collected':'mean', 'Balance': 'mean'}).reset_index(drop=True)
client_avgs.columns = ['Patient Guid', 'Average Charged', 'Average Collected', 'Average Balance']
client_details = pd.merge(client_details, client_avgs, how='left', on='Patient Guid')
client_details['Average Charged'] = round(client_details['Average Charged'], 2)
client_details['Average Collected'] = round(client_details['Average Collected'], 2)

client_details.head()

,Patient Guid,# of Sessions,Total Charged,Total Collected,Total Outstanding,Average Charged,Average Collected,Average Balance
0,29276-100,2,320.0,320.0,0.0,160.00,160.00,0.0
1,29276-13,3,409.5,409.5,0.0,136.50,136.50,0.0
2,29276-23,7,975.0,975.0,0.0,139.29,139.29,0.0
3,29276-26,3,475.0,475.0,0.0,158.33,158.33,0.0
4,29276-38,4,635.0,635.0,0.0,158.75,158.75,0.0


In [17]:
client_details.describe()

,# of Sessions,Total Charged,Total Collected,Total Outstanding,Average Charge,Average Collected,Average Balance
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,3.200000,481.821429,463.364286,18.457143,150.647714,143.733714,6.914286
std,1.605139,254.245883,270.345308,71.264356,20.156972,33.676681,25.663527
min,1.000000,157.500000,62.750000,0.000000,84.000000,20.920000,0.000000
25%,2.000000,315.000000,315.000000,0.000000,157.500000,153.750000,0.000000
50%,3.000000,420.000000,409.500000,0.000000,158.330000,158.330000,0.000000
75%,4.000000,569.500000,569.500000,0.000000,158.750000,158.750000,0.000000
max,7.000000,1110.000000,1110.000000,386.000000,168.000000,168.000000,128.666667


In [7]:
fig1 = px.histogram(df, x='Total', title='Sliding Scale Distribution by Session Count', nbins=50)
fig1.update_layout(xaxis_title='Amount Charged', yaxis_title='Session Count')
fig1.update_xaxes(range=[0, None])
fig1.show()

In [34]:
fig2 = px.histogram(client_details, x='Average Charged', title='Sliding Scale Distribution by Client Count', nbins=50)
fig2.update_layout(xaxis_title='Amount Charged', yaxis_title='Client Count')
fig2.update_xaxes(range=[0, None])
fig2.show()

In [19]:
days_worked = df['Purchase Date'].nunique()
avg_sessions_per_day = annual_details.loc[0]['# of Sessions'] / days_worked
print('Total Days Worked: ', days_worked)
print('Average number of sessions per day worked: ', avg_sessions_per_day)


Total Days Worked:  28
Average number of sessions per day worked:  4.0


# March 2024 - Feb 2025 Review

In [59]:
df = pd.read_csv('Sales_20240301_20250228.csv')
df.columns = df.columns.str.strip()
df.drop(columns=['Invoice Date', 'Unnamed: 4', 'Subtotal', 'Location', 'Staff Member', 'Income Category', 'Payer', 'Details'], inplace=True)
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'], format='%Y-%m-%d')

monthly_details = df.groupby(df['Purchase Date'].dt.to_period('M')).agg({'Invoice #': 'count', 'Patient Guid': pd.Series.nunique, 'Total': 'sum', 'Collected':'sum', 'Balance': 'sum'}).reset_index()
monthly_details['Total_avg'] = round(monthly_details['Total'] / monthly_details['Invoice #'], 2)
monthly_details['Collected_avg'] = round(monthly_details['Collected'] / monthly_details['Invoice #'], 2)
monthly_details['Balance_avg'] = round(monthly_details['Balance'] / monthly_details['Invoice #'], 2)
monthly_details.columns = ['Month', '# of Sessions', 'Unique Clients', 'Total Charged', 'Total Collected', 'Toal Outstanding', 'Average Charged', 'Average Collected', 'Average Outstanding']

annual_details = df.groupby(df['Purchase Date'].dt.to_period('Y')).agg({'Invoice #': 'count', 'Patient Guid': pd.Series.nunique, 'Total': 'sum', 'Collected':'sum', 'Balance': 'sum'}).reset_index()
annual_details['Total_avg'] = round(annual_details['Total'] / annual_details['Invoice #'], 2)
annual_details['Collected_avg'] = round(annual_details['Collected'] / annual_details['Invoice #'], 2)
annual_details['Balance_avg'] = round(annual_details['Balance'] / annual_details['Invoice #'], 2)
annual_details.columns = ['Year', '# of Sessions', 'Unique Clients', 'Total Charged', 'Total Collected', 'Toal Outstanding', 'Average Charged', 'Average Collected', 'Average Outstanding']

client_details = df.groupby('Patient Guid', as_index=False).agg({'Invoice #': 'count', 'Total': 'sum', 'Collected':'sum', 'Balance': 'sum'}).reset_index(drop=True)
client_details.columns = ['Patient Guid', '# of Sessions', 'Total Charged', 'Total Collected', 'Total Outstanding']
client_avgs = df.groupby('Patient Guid', as_index=False).agg({'Total': 'mean', 'Collected':'mean', 'Balance': 'mean'}).reset_index(drop=True)
client_avgs.columns = ['Patient Guid', 'Average Charged', 'Average Collected', 'Average Balance']
client_details = pd.merge(client_details, client_avgs, how='left', on='Patient Guid')
client_details['Average Charged'] = round(client_details['Average Charged'], 2)
client_details['Average Collected'] = round(client_details['Average Collected'], 2)

In [ ]:
days_worked = df['Purchase Date'].nunique()
avg_sessions_per_day = round(len(df) / days_worked, 2)
avg_charged = df['Total'].mean()
total_clients = df['Patient Guid'].nunique()
total_revenue = df['Collected'].sum()

print('Total Days Clients Seen: ', days_worked)
# add total days worked = client days plus a friday for each week any clients were seen
print('Total Sessions: ', len(df))
print('Average number of sessions per day worked: ', avg_sessions_per_day)
print('Total unique clients: ', total_clients)
print('Average rate charged: ', locale.currency(avg_charged, grouping=True))
print('Total revenue: ', locale.currency(total_revenue, grouping=True))

Total Days Clients Seen:  185
Total Sessions:  745
Average number of sessions per day worked:  4.03
Total unique clients:  48
Average rate charged:  $147.61
Total revenue:  $109,740.50


In [60]:
monthly_details

,Month,# of Sessions,Unique Clients,Total Charged,Total Collected,Toal Outstanding,Average Charged,Average Collected,Average Outstanding
0,2024-03,67,35,9707.00,9707.00,0.0,144.88,144.88,0.00
1,2024-04,67,34,9991.00,9991.00,0.0,149.12,149.12,0.00
2,2024-05,59,33,8515.00,8515.00,0.0,144.32,144.32,0.00
3,2024-06,58,33,8571.75,8571.75,0.0,147.79,147.79,0.00
4,2024-07,63,32,9266.75,9266.75,0.0,147.09,147.09,0.00
5,2024-08,65,32,9522.00,9522.00,0.0,146.49,146.49,0.00
6,2024-09,47,30,6899.00,6899.00,0.0,146.79,146.79,0.00
7,2024-10,81,36,11900.75,11900.75,0.0,146.92,146.92,0.00
8,2024-11,62,31,9427.25,9427.25,0.0,152.05,152.05,0.00
9,2024-12,48,29,7135.00,7135.00,0.0,148.65,148.65,0.00


In [61]:
annual_details

,Year,# of Sessions,Unique Clients,Total Charged,Total Collected,Toal Outstanding,Average Charged,Average Collected,Average Outstanding
0,2024,617,47,90935.5,90935.5,0.0,147.38,147.38,0.00
1,2025,128,36,19031.0,18805.0,226.0,148.68,146.91,1.77


In [62]:
fig1 = px.histogram(df, x='Total', title='Sliding Scale Distribution by Session Count', nbins=50)
fig1.update_layout(xaxis_title='Amount Charged', yaxis_title='Session Count')
fig1.update_xaxes(range=[0, None])
fig1.show()

# Frequency calc

In [26]:
df = df.sort_values(by=['Patient Guid', 'Purchase Date'])
df['Days Between'] = df.groupby('Patient Guid')['Purchase Date'].diff().dt.days

In [27]:
df

,Purchase Date,Patient Guid,Item,Invoice #,Status,Total,Collected,Balance,Days Between
13,2025-02-05,29276-100,Initial Individual Counselling Session (Video),1470-P02,paid,160.0,160.0,0.0,NaN
81,2025-03-06,29276-100,Individual Counselling Session (Video),1538-P01,paid,160.0,160.0,0.0,29.0
16,2025-02-06,29276-13,Individual Counselling Session (Video),1473-P01,paid,136.5,136.5,0.0,NaN
63,2025-02-26,29276-13,Individual Counselling Session (Video),1520-P01,paid,136.5,136.5,0.0,20.0
80,2025-03-06,29276-13,Individual Counselling Session (Video),1537-P01,paid,136.5,136.5,0.0,8.0
...,...,...,...,...,...,...,...,...,...
76,2025-03-05,29276-93,Individual Counselling Session (Video),1533-P01,paid,160.0,160.0,0.0,23.0
102,2025-03-26,29276-93,Individual Counselling Session (Video),1559-P01,paid,160.0,160.0,0.0,21.0
6,2025-02-04,29276-98,Individual Counselling Session (Video),1463-P01,paid,140.0,140.0,0.0,NaN
38,2025-02-18,29276-98,Individual Counselling Session (In-Person),1495-P01,paid,140.0,140.0,0.0,14.0


In [37]:
frequency_df = df.groupby('Patient Guid')['Days Between'].mean().reset_index()
frequency_df.columns = ['Patient Guid', 'Avg Days Between Sessions']
frequency_df

,Patient Guid,Avg Days Between Sessions
0,29276-100,29.000000
1,29276-13,14.000000
2,29276-23,8.166667
3,29276-26,24.500000
4,29276-38,14.000000
5,29276-39,14.000000
6,29276-40,14.000000
7,29276-42,14.000000
8,29276-47,14.000000
9,29276-48,16.000000


In [ ]:
frequency_df['Frequency'] = frequency_df['Avg Days Between Sessions'].apply(classify_frequency)

In [39]:
frequency_df

,Patient Guid,Avg Days Between Sessions,Frequency
0,29276-100,29.000000,Monthly
1,29276-13,14.000000,Bi-weekly
2,29276-23,8.166667,Weekly
3,29276-26,24.500000,Monthly
4,29276-38,14.000000,Bi-weekly
5,29276-39,14.000000,Bi-weekly
6,29276-40,14.000000,Bi-weekly
7,29276-42,14.000000,Bi-weekly
8,29276-47,14.000000,Bi-weekly
9,29276-48,16.000000,Bi-weekly


In [40]:
frequency_df = pd.merge(frequency_df, client_details[['Patient Guid', 'Average Charged']], how='left', on='Patient Guid')
frequency_df.head(10)

,Patient Guid,Avg Days Between Sessions,Frequency,Average Charged
0,29276-100,29.000000,Monthly,160.00
1,29276-13,14.000000,Bi-weekly,136.50
2,29276-23,8.166667,Weekly,139.29
3,29276-26,24.500000,Monthly,158.33
4,29276-38,14.000000,Bi-weekly,158.75
5,29276-39,14.000000,Bi-weekly,84.00
6,29276-40,14.000000,Bi-weekly,158.50
7,29276-42,14.000000,Bi-weekly,158.33
8,29276-47,14.000000,Bi-weekly,157.50
9,29276-48,16.000000,Bi-weekly,158.75


In [41]:
def annual_forecast(row):
    if row['Frequency'] == 'Weekly':
        return row['Average Charged'] * 52
    elif row['Frequency'] == 'Bi-weekly':
        return row['Average Charged'] * 26
    elif row['Frequency'] == 'Monthly':
        return row['Average Charged'] * 12
    else:
        return 0

In [42]:
projection = frequency_df.drop(columns='Avg Days Between Sessions')
projection['Annual Revenue'] = projection.apply(annual_forecast, axis=1)
projection

,Patient Guid,Frequency,Average Charged,Annual Revenue
0,29276-100,Monthly,160.00,1920.00
1,29276-13,Bi-weekly,136.50,3549.00
2,29276-23,Weekly,139.29,7243.08
3,29276-26,Monthly,158.33,1899.96
4,29276-38,Bi-weekly,158.75,4127.50
5,29276-39,Bi-weekly,84.00,2184.00
6,29276-40,Bi-weekly,158.50,4121.00
7,29276-42,Bi-weekly,158.33,4116.58
8,29276-47,Bi-weekly,157.50,4095.00
9,29276-48,Bi-weekly,158.75,4127.50
